In [5]:
import pandas as pd
import numpy as np

# Carregando os dados
caminho = "C:\\Users\\natal\\OneDrive\\Área de Trabalho\\Estudo Python\\SistemaRecomendacao\\ml-latest-small"
caminho_arquivo = caminho + "\\movies.csv"
filmes = pd.read_csv(caminho_arquivo)
filmes.columns = ["filmeId", "titulo", "genero"]
filmes = filmes.set_index("filmeId")

caminho_arquivo = caminho + "\\ratings.csv"
notas = pd.read_csv(caminho_arquivo)
notas.columns = ["usuarioId", "filmeId", "nota", "momento"]

In [8]:
len(filmes)

9742

In [25]:
# Calculando o total de votos por filme e a média das notas por filme
total_de_votos = notas["filmeId"].value_counts()
filmes["total_votos"] = total_de_votos

media_das_notas = notas.groupby("filmeId").mean()["nota"]
filmes["media_das_notas"] = media_das_notas


filmes_com_mais_de_cinquenta_votos = filmes.query("total_votos> 50")


In [30]:
def notas_do_usuario(usuario):
    notas_do_usuario = notas.query("usuarioId==%d" % usuario)
    notas_do_usuario = notas_do_usuario[["filmeId", "nota"]].set_index(["filmeId"])
    return notas_do_usuario

In [31]:
def distancia_de_vetores(a,b):
    return np.linalg.norm(a - b)

In [32]:
def distancia_de_usuarios(usuarioId1, usuarioId2):
    notas1 = notas_do_usuario(usuarioId1)
    notas2 = notas_do_usuario(usuarioId2)
    diferencas = notas1.join(notas2, lsuffix = "_esquerda", rsuffix = "_direita").dropna()
    distancia = distancia_de_vetores(diferencas['nota_esquerda'], diferencas['nota_direita'])
    return [usuarioId1, usuarioId2, distancia]

In [35]:
def distancia_de_todos(voce_id, numero_de_usuarios_a_analisar  = None):
    todos_os_usuarios = notas['usuarioId'].unique()
    if numero_de_usuarios_a_analisar:
        todos_os_usuarios = todos_os_usuarios[:numero_de_usuarios_a_analisar ]
    distancias = [distancia_de_usuarios(voce_id, usuario_Id) for usuario_Id in todos_os_usuarios]
    distancias = list(filter(None, distancias))
    distancias = pd.DataFrame(distancias, columns = ["você", "outra_pessoa", "distancia"])
    return distancias

In [43]:
notas = notas.set_index("filmeId").loc[filmes_com_mais_de_cinquenta_votos.index]

In [44]:
def knn(voce_id, k_mais_proximos=10, numero_de_usuarios_a_analisar = None):
  distancias = distancia_de_todos(voce_id, numero_de_usuarios_a_analisar = numero_de_usuarios_a_analisar)
  distancias = distancias.sort_values("distancia")
  distancias = distancias.set_index("outra_pessoa").drop(voce_id, errors='ignore')
  return distancias.head(k_mais_proximos)

In [45]:
def sugere_para(voce, k_mais_proximos = 10, numero_de_usuarios_a_analisar = None):
  notas_de_voce = notas_do_usuario(voce)
  filmes_que_voce_ja_viu = notas_de_voce.index

  similares = knn(voce, k_mais_proximos = k_mais_proximos, numero_de_usuarios_a_analisar = numero_de_usuarios_a_analisar)
  usuarios_similares = similares.index
  notas_dos_similares = notas.set_index("usuarioId").loc[usuarios_similares]
  recomendacoes = notas_dos_similares.groupby("filmeId").mean()[["nota"]]
  aparicoes = notas_dos_similares.groupby("filmeId").count()[['nota']]
  
  filtro_minimo = k_mais_proximos / 2
  recomendacoes = recomendacoes.join(aparicoes, lsuffix="_media_dos_usuarios", rsuffix="_aparicoes_nos_usuarios")
  recomendacoes = recomendacoes.query("nota_aparicoes_nos_usuarios >= %.2f" % filtro_minimo)  
  recomendacoes = recomendacoes.sort_values("nota_media_dos_usuarios", ascending=False)
  recomendacoes = recomendacoes.drop(filmes_que_voce_ja_viu,errors='ignore')
  return recomendacoes.join(filmes)

In [46]:
sugere_para(1)

KeyError: "['filmeId'] not in index"